In [ ]:
import pandas as pd
import numpy as np
import math
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool
from decimal import Decimal
import datetime
import os
#pd.set_option('display.max_columns', None)
#DB Details
connection_type = "mssql+pyodbc"
dataportal_db_user = "dataportal"
dataportal_db_password = "FatShamingMarc"
dataportal_db_ip = "10.8.4.35"
db_port = "1433"
db_prod_name = "dataportal_prod"
db_driver = "ODBC+DRIVER+17+for+SQL+Server"

In [ ]:
def create_connection_string(connection_type, user, password, host, port, database_name, driver):
    return (
        "{connection_type}://{user}:{password}@{host}:{port}/{database_name}"
        "?driver={driver}").format(
        connection_type=connection_type,
        user=user,
        password=password,
        host=host,
        port=port,
        database_name=database_name,
        driver=driver)

In [ ]:
def mapOfflineEventsDataframe(df, filename):
    df.loc[df.PADD_REG == 'I', 'PADD_REG'] = "PAD District 1"
    df.loc[df.PADD_REG == 'II', 'PADD_REG'] = "PAD District 2"
    df.loc[df.PADD_REG == 'III', 'PADD_REG'] = "PAD District 3"
    df.loc[df.PADD_REG == 'IV', 'PADD_REG'] = "PAD District 4"
    df.loc[df.PADD_REG == 'V', 'PADD_REG'] = "PAD District 5"
    df['ORIGIN_FILE'] = filename

In [ ]:
#Define columns we're interested in from the file
columns=['RELEASE_DT', 'EVENT_ID', 'UNIT_NAME', 'UNIT_ID', 'AREA_ID', 'AREA_NAME', 'OWNER_NAME', 'PLANT_ID', 'PLANT_NAME', 
         'U_STATUS', 'UTYPE_DESC', 'U_CAPACITY', 'CAP_OFFLINE', 'START_DATE', 'END_DATE', 'PRECISION', 'EVENT_TYPE', 
         'E_STATUS', 'E_CAUSE', 'E_DURATION', 'PADD_REG', 'COMMENTS']

In [ ]:
renamed_columns={"RELEASE_DT": "PUBLISH_DATE", "U_STATUS":"UNIT_STATUS", "U_CAPACITY":"CAPACITY", "CAP_OFFLINE":"CAPACITY_OFFLINE", "START_DATE": "EVENT_START", "END_DATE": "EVENT_END", 
                 "PRECISION": "DATE_PRECISION", "E_STATUS": "EVENT_STATUS", "E_CAUSE": "EVENT_CAUSE", "E_DURATION" : "EVENT_DURATION", "PADD_REG": "PAD_DISTRICT"}

In [ ]:
filename = 'OfflineEvents20200825.csv'
path = 'C:\\Temp\\IIR\\'
filepath = path + filename

In [ ]:
connection_string = create_connection_string(connection_type,dataportal_db_user,dataportal_db_password,dataportal_db_ip,db_port,db_prod_name, db_driver)
engine = create_engine(connection_string, poolclass=StaticPool)

In [ ]:
#Manual Run
filepath = path + filename
df = pd.read_csv(filepath, low_memory=False)[columns]
mapOfflineEventsDataframe(df, filename)
df = df.rename(columns = renamed_columns)

In [ ]:
# Insert to SQL
df.to_sql('data_iir_offlineevents', connection_string, if_exists='append', index=False,
        dtype={ 'PUBLISH_DATE' :   sa.DateTime(),
                'EVENT_ID' :       sa.types.INTEGER(),
                'UNIT_NAME' :     sa.types.VARCHAR(100),
                'UNIT_ID' :       sa.types.INTEGER(),
                'AREA_ID' :       sa.types.INTEGER(),
                'AREA_NAME' :     sa.types.VARCHAR(100),
                'OWNER_NAME' :     sa.types.VARCHAR(100),
                'PLANT_ID' :       sa.types.INTEGER(),
                'PLANT_NAME' :     sa.types.VARCHAR(100),
                'UNIT_STATUS' : sa.types.VARCHAR(30),
                'UTYPE_DESC' : sa.types.VARCHAR(100),
                'CAPACITY' : sa.types.INTEGER(),
                'CAPACITY_OFFLINE' : sa.types.INTEGER(),
                'EVENT_START' :  sa.DateTime(),
                'EVENT_END' :  sa.DateTime(),
                'DATE_PRECISION' : sa.types.VARCHAR(16) ,
                'EVENT_TYPE' : sa.types.VARCHAR(30),         
                'EVENT_STATUS' : sa.types.VARCHAR(30),
                'EVENT_CAUSE' : sa.types.VARCHAR(65),
                'EVENT_DURATION' : sa.types.INTEGER(),
                'PAD_DISTRICT' : sa.types.VARCHAR(65),
                'COMMENTS' : sa.types.TEXT ,
                'ORIGIN_FILE' : sa.types.VARCHAR(65)})